# **📌 Packages**

In [1]:
# 기본
import pandas as pd
import numpy as np

In [2]:
# 시각화
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

## 한글 처리
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17,8

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# **📌 대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [4]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [5]:
import pandas as pd
from datetime import datetime

train = pd.read_csv('/Users/euijinlee/KDT_DATA/Project_1/data/train.csv', encoding = "utf-8", low_memory=False)
test = pd.read_csv('/Users/euijinlee/KDT_DATA/Project_1/data/test.csv', encoding = "utf-8", low_memory=False)

display(train.head())
display(test.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [6]:
display(f"train : {train.iloc[0]['사고일시']} ~ {train.iloc[-1]['사고일시']}")
display(f"test : {test.iloc[0]['사고일시']} ~ {test.iloc[-1]['사고일시']}")

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

### Data Info

In [7]:
#'사고일시' 데이터 타입을 object -> datetime으로  바꾸기
train['사고일시'] = pd.to_datetime(train['사고일시'])
test['사고일시'] = pd.to_datetime(test['사고일시'])

## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

### **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [8]:
import pandas as pd
from datetime import datetime

train_org = pd.read_csv('/Users/euijinlee/KDT_DATA/Project_1/data/train.csv',encoding = "utf-8", low_memory=False)
test_org = pd.read_csv('/Users/euijinlee/KDT_DATA/Project_1/data/test.csv', encoding = "utf-8", low_memory=False)

display(train_org.head())
display(test_org.tail())

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,ACCIDENT_50571,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


In [9]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

### **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [10]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

### **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [11]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

# **📌 추가 전처리**

## **train, test 컬럼 수 맞추기**

test에 있는 컬럼들만 사용하여 train 데이터를 구성했습니다.

In [12]:
train_df= train_df[['ID','요일','기상상태','노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구','동', '도로형태1', '도로형태2']]
train_df

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,2019,1,1,4,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,금요일,맑음,건조,차대차,2021,12,31,19,대구광역시,수성구,수성동3가,교차로,교차로안
39605,ACCIDENT_39605,금요일,맑음,건조,차대차,2021,12,31,19,대구광역시,달서구,상인동,단일로,기타
39606,ACCIDENT_39606,금요일,맑음,건조,차대차,2021,12,31,21,대구광역시,달서구,월성동,교차로,교차로안
39607,ACCIDENT_39607,금요일,맑음,건조,차대차,2021,12,31,22,대구광역시,달서구,장동,기타,기타


## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

In [13]:
display(train_df.head())
display(test_df.head())

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,ACCIDENT_00003,화요일,맑음,건조,차대차,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,ACCIDENT_00004,화요일,맑음,건조,차대차,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,ACCIDENT_39611,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,ACCIDENT_39612,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,ACCIDENT_39613,토요일,맑음,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안


In [14]:
display(train_df.info())
display(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   요일      39609 non-null  object
 2   기상상태    39609 non-null  object
 3   노면상태    39609 non-null  object
 4   사고유형    39609 non-null  object
 5   연       39609 non-null  int64 
 6   월       39609 non-null  int64 
 7   일       39609 non-null  int64 
 8   시간      39609 non-null  int64 
 9   도시      39609 non-null  object
 10  구       39609 non-null  object
 11  동       39609 non-null  object
 12  도로형태1   39609 non-null  object
 13  도로형태2   39609 non-null  object
dtypes: int64(4), object(10)
memory usage: 4.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   요일      10963 non-null  object
 2   기상상태    10963 non-null  object
 3   노면상태    10963 non-null  object
 4   사고유형    10963 non-null  object
 5   연       10963 non-null  int64 
 6   월       10963 non-null  int64 
 7   일       10963 non-null  int64 
 8   시간      10963 non-null  int64 
 9   도시      10963 non-null  object
 10  구       10963 non-null  object
 11  동       10963 non-null  object
 12  도로형태1   10963 non-null  object
 13  도로형태2   10963 non-null  object
dtypes: int64(4), object(10)
memory usage: 1.2+ MB


None

# **📌 인코딩 (Label & One-Hot)**

## **0. 데이터 초기화 (data_label_enco, data_onehot_enco)**

In [15]:
# 전처리를 위해 train과 test 데이터 합치기
data = pd.concat([train_df, test_df], sort=False)
data_label_enco = data.copy()
data_onehot_enco = data.copy()

## **1. 노면상태, 사고유형 인코딩**

### **Label Encoding - 노면상태, 사고유형**

In [16]:
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# "사고유형" 컬럼 Label Encoding
data_label_enco['사고유형'] = label_encoder.fit_transform(data_label_enco['사고유형'])

# "노면상태" 컬럼 Label Encoding
data_label_enco['노면상태'] = label_encoder.fit_transform(data_label_enco['노면상태'])

# 결과 확인
print(data_label_enco[['사고유형', '노면상태']])
data_label_enco.info()

       사고유형  노면상태
0         0     0
1         0     0
2         0     0
3         1     0
4         1     0
...     ...   ...
10958     1     0
10959     1     0
10960     1     0
10961     1     0
10962     1     0

[50572 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  object
 13  도로형태2   50572 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.8+ MB


### **One-Hot Encoding - 노면상태, 사고유형**

In [17]:
# "사고유형" 및 "노면상태" 컬럼을 원-핫 인코딩
data_onehot_enco = pd.get_dummies(data, columns=['사고유형', '노면상태'])

## **2. 도로형태1, 도로형태2 인코딩**

### **Label Encoding - 도로형태1**

In [18]:
le = LabelEncoder()
le.fit(data_label_enco['도로형태1'])

LabelEncoder()

In [19]:
le.transform(data_label_enco['도로형태1'])

array([2, 2, 2, ..., 2, 0, 0])

In [20]:
label_data = data.copy()

In [21]:
data_label_enco['도로형태1'] = le.transform(data_label_enco['도로형태1'])

In [22]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int64 
 13  도로형태2   50572 non-null  object
dtypes: int64(7), object(7)
memory usage: 5.8+ MB


### **Label Encoding - 도로형태2**

In [23]:
le = LabelEncoder()
le.fit(data_label_enco['도로형태2'])

LabelEncoder()

In [24]:
le.transform(data_label_enco['도로형태2'])

array([5, 5, 5, ..., 5, 2, 2])

In [25]:
data_label_enco['도로형태2'] = le.transform(data_label_enco['도로형태2'])

In [26]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int64 
 13  도로형태2   50572 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 5.8+ MB


### **One-Hot Encoding - 도로형태1 & 도로형태2**

In [27]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['도로형태1', '도로형태2'])

In [28]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   연                50572 non-null  int64 
 4   월                50572 non-null  int64 
 5   일                50572 non-null  int64 
 6   시간               50572 non-null  int64 
 7   도시               50572 non-null  object
 8   구                50572 non-null  object
 9   동                50572 non-null  object
 10  사고유형_차대사람        50572 non-null  bool  
 11  사고유형_차대차         50572 non-null  bool  
 12  사고유형_차량단독        50572 non-null  bool  
 13  노면상태_건조          50572 non-null  bool  
 14  노면상태_기타          50572 non-null  bool  
 15  노면상태_서리/결빙       50572 non-null  bool  
 16  노면상태_적설          50572 non-null  bool  
 17  노면상태_젖음/습기       50572 non-null  boo

## **3. 구, 동 인코딩**

### **Label Encoding - 구**

In [29]:
label_encoder_gu = LabelEncoder()

In [30]:
data_label_enco['구'] = label_encoder_gu.fit_transform(data['구'])

In [31]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int64 
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int64 
 13  도로형태2   50572 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 5.8+ MB


### **One-Hot Encoding - 구**

In [32]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['구'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   연                50572 non-null  int64 
 4   월                50572 non-null  int64 
 5   일                50572 non-null  int64 
 6   시간               50572 non-null  int64 
 7   도시               50572 non-null  object
 8   동                50572 non-null  object
 9   사고유형_차대사람        50572 non-null  bool  
 10  사고유형_차대차         50572 non-null  bool  
 11  사고유형_차량단독        50572 non-null  bool  
 12  노면상태_건조          50572 non-null  bool  
 13  노면상태_기타          50572 non-null  bool  
 14  노면상태_서리/결빙       50572 non-null  bool  
 15  노면상태_적설          50572 non-null  bool  
 16  노면상태_젖음/습기       50572 non-null  bool  
 17  노면상태_침수          50572 non-null  boo

### **(Label Encoding - 동) => 동 정보 일단 미적용**

In [33]:
# label_encoder_gu = LabelEncoder()

In [34]:
# data_label_enco['동'] = label_encoder_gu.fit_transform(data['동'])

In [35]:
# data_label_enco.info()

### **(One-Hot Encoding - 동) => 동 정보 일단 미적용**

In [36]:
# data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['동'])
# data_onehot_enco.info()

## **4. 연,월,시 인코딩**
- 이미 연도, 월, 시간은 숫자형 데이터로 되어 있기 때문에 Label Encoding을 적용 필요 X => One-Hot Encoding만 진행

### **One-Hot Encoding - 연**

In [37]:
# 연도의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['연'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 44 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   월                50572 non-null  int64 
 4   일                50572 non-null  int64 
 5   시간               50572 non-null  int64 
 6   도시               50572 non-null  object
 7   동                50572 non-null  object
 8   사고유형_차대사람        50572 non-null  bool  
 9   사고유형_차대차         50572 non-null  bool  
 10  사고유형_차량단독        50572 non-null  bool  
 11  노면상태_건조          50572 non-null  bool  
 12  노면상태_기타          50572 non-null  bool  
 13  노면상태_서리/결빙       50572 non-null  bool  
 14  노면상태_적설          50572 non-null  bool  
 15  노면상태_젖음/습기       50572 non-null  bool  
 16  노면상태_침수          50572 non-null  bool  
 17  도로형태1_교차로        50572 non-null  boo

### **One-Hot Encoding - 월**

In [38]:
# 월의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['월'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   일                50572 non-null  int64 
 4   시간               50572 non-null  int64 
 5   도시               50572 non-null  object
 6   동                50572 non-null  object
 7   사고유형_차대사람        50572 non-null  bool  
 8   사고유형_차대차         50572 non-null  bool  
 9   사고유형_차량단독        50572 non-null  bool  
 10  노면상태_건조          50572 non-null  bool  
 11  노면상태_기타          50572 non-null  bool  
 12  노면상태_서리/결빙       50572 non-null  bool  
 13  노면상태_적설          50572 non-null  bool  
 14  노면상태_젖음/습기       50572 non-null  bool  
 15  노면상태_침수          50572 non-null  bool  
 16  도로형태1_교차로        50572 non-null  bool  
 17  도로형태1_기타         50572 non-null  boo

### **One-Hot Encoding - 시**

In [39]:
# 시간의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['시간'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 78 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   일                50572 non-null  int64 
 4   도시               50572 non-null  object
 5   동                50572 non-null  object
 6   사고유형_차대사람        50572 non-null  bool  
 7   사고유형_차대차         50572 non-null  bool  
 8   사고유형_차량단독        50572 non-null  bool  
 9   노면상태_건조          50572 non-null  bool  
 10  노면상태_기타          50572 non-null  bool  
 11  노면상태_서리/결빙       50572 non-null  bool  
 12  노면상태_적설          50572 non-null  bool  
 13  노면상태_젖음/습기       50572 non-null  bool  
 14  노면상태_침수          50572 non-null  bool  
 15  도로형태1_교차로        50572 non-null  bool  
 16  도로형태1_기타         50572 non-null  bool  
 17  도로형태1_단일로        50572 non-null  boo

## **5. 요일, 기상상태 인코딩**

### **Label Encoding - 요일**
- 0 = 금요일 / 1 = 목요일 / 2 = 수요일 / 3 = 월요일 / 4 = 일요일 / 5 = 토요일 / 6 = 화요일

In [40]:
# 인코딩된 값 삽입위해 기존 '요일' 컬럼 삭제
data_label_enco = data_label_enco.drop(data_label_enco.columns[1], axis=1).copy()

In [41]:
# 요일 컬럼 Label Encoding 후 컬럼 순서 다시 정렬
encoder = LabelEncoder()
encoder.fit(data['요일'])
data_label_enco['요일'] = encoder.transform(data['요일'])
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   기상상태    50572 non-null  object
 2   노면상태    50572 non-null  int64 
 3   사고유형    50572 non-null  int64 
 4   연       50572 non-null  int64 
 5   월       50572 non-null  int64 
 6   일       50572 non-null  int64 
 7   시간      50572 non-null  int64 
 8   도시      50572 non-null  object
 9   구       50572 non-null  int64 
 10  동       50572 non-null  object
 11  도로형태1   50572 non-null  int64 
 12  도로형태2   50572 non-null  int64 
 13  요일      50572 non-null  int64 
dtypes: int64(10), object(4)
memory usage: 5.8+ MB


In [42]:
data_label_enco = data_label_enco[['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2']]
display(data_label_enco.head())
print('인코딩 변환값 :', set(data_label_enco['요일']))

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,6,맑음,0,0,2019,1,1,0,대구광역시,7,대신동,2,5
1,ACCIDENT_00001,6,흐림,0,0,2019,1,1,0,대구광역시,1,감삼동,2,5
2,ACCIDENT_00002,6,맑음,0,0,2019,1,1,1,대구광역시,6,두산동,2,5
3,ACCIDENT_00003,6,맑음,0,1,2019,1,1,2,대구광역시,4,복현동,2,5
4,ACCIDENT_00004,6,맑음,0,1,2019,1,1,4,대구광역시,3,신암동,2,5


인코딩 변환값 : {0, 1, 2, 3, 4, 5, 6}


### **One-Hot Encoding - 요일**

In [43]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['요일'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 84 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   기상상태             50572 non-null  object
 2   일                50572 non-null  int64 
 3   도시               50572 non-null  object
 4   동                50572 non-null  object
 5   사고유형_차대사람        50572 non-null  bool  
 6   사고유형_차대차         50572 non-null  bool  
 7   사고유형_차량단독        50572 non-null  bool  
 8   노면상태_건조          50572 non-null  bool  
 9   노면상태_기타          50572 non-null  bool  
 10  노면상태_서리/결빙       50572 non-null  bool  
 11  노면상태_적설          50572 non-null  bool  
 12  노면상태_젖음/습기       50572 non-null  bool  
 13  노면상태_침수          50572 non-null  bool  
 14  도로형태1_교차로        50572 non-null  bool  
 15  도로형태1_기타         50572 non-null  bool  
 16  도로형태1_단일로        50572 non-null  bool  
 17  도로형태1_미분류        50572 non-null  boo

In [44]:
# 요일 컬럼 순서 변경 (75번부터 80번 컬럼)
desired_order = ['요일_월요일', '요일_화요일', '요일_수요일', '요일_목요일', '요일_금요일', '요일_토요일', '요일_일요일']
data_onehot_enco = data_onehot_enco[ [col for col in data_onehot_enco.columns if col not in desired_order] + desired_order]

In [45]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 84 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   기상상태             50572 non-null  object
 2   일                50572 non-null  int64 
 3   도시               50572 non-null  object
 4   동                50572 non-null  object
 5   사고유형_차대사람        50572 non-null  bool  
 6   사고유형_차대차         50572 non-null  bool  
 7   사고유형_차량단독        50572 non-null  bool  
 8   노면상태_건조          50572 non-null  bool  
 9   노면상태_기타          50572 non-null  bool  
 10  노면상태_서리/결빙       50572 non-null  bool  
 11  노면상태_적설          50572 non-null  bool  
 12  노면상태_젖음/습기       50572 non-null  bool  
 13  노면상태_침수          50572 non-null  bool  
 14  도로형태1_교차로        50572 non-null  bool  
 15  도로형태1_기타         50572 non-null  bool  
 16  도로형태1_단일로        50572 non-null  bool  
 17  도로형태1_미분류        50572 non-null  boo

### **Label Encoding - 기상상태**

In [46]:
encoder_weather = LabelEncoder()
encoder_weather.fit(data['기상상태'])
data_label_enco['기상상태'] = encoder_weather.transform(data['기상상태'])

display(data_label_enco)
print('인코딩 변환값 : ', set(data_label_enco['기상상태']))

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,6,2,0,0,2019,1,1,0,대구광역시,7,대신동,2,5
1,ACCIDENT_00001,6,5,0,0,2019,1,1,0,대구광역시,1,감삼동,2,5
2,ACCIDENT_00002,6,2,0,0,2019,1,1,1,대구광역시,6,두산동,2,5
3,ACCIDENT_00003,6,2,0,1,2019,1,1,2,대구광역시,4,복현동,2,5
4,ACCIDENT_00004,6,2,0,1,2019,1,1,4,대구광역시,3,신암동,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,5,2,0,1,2022,12,31,18,대구광역시,0,대명동,2,9
10959,ACCIDENT_50568,5,2,0,1,2022,12,31,18,대구광역시,6,시지동,2,5
10960,ACCIDENT_50569,5,2,0,1,2022,12,31,20,대구광역시,6,연호동,2,5
10961,ACCIDENT_50570,5,2,0,1,2022,12,31,20,대구광역시,6,범물동,0,2


인코딩 변환값 :  {0, 1, 2, 3, 4, 5}


In [47]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  int64 
 2   기상상태    50572 non-null  int64 
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int64 
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int64 
 13  도로형태2   50572 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 5.8+ MB


### **One-Hot Encoding - 기상상태**

In [48]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['기상상태'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   일                50572 non-null  int64 
 2   도시               50572 non-null  object
 3   동                50572 non-null  object
 4   사고유형_차대사람        50572 non-null  bool  
 5   사고유형_차대차         50572 non-null  bool  
 6   사고유형_차량단독        50572 non-null  bool  
 7   노면상태_건조          50572 non-null  bool  
 8   노면상태_기타          50572 non-null  bool  
 9   노면상태_서리/결빙       50572 non-null  bool  
 10  노면상태_적설          50572 non-null  bool  
 11  노면상태_젖음/습기       50572 non-null  bool  
 12  노면상태_침수          50572 non-null  bool  
 13  도로형태1_교차로        50572 non-null  bool  
 14  도로형태1_기타         50572 non-null  bool  
 15  도로형태1_단일로        50572 non-null  bool  
 16  도로형태1_미분류        50572 non-null  bool  
 17  도로형태1_주차장        50572 non-null  boo

## **6. 인코딩 확인**

In [49]:
data_onehot_enco.columns

Index(['ID', '일', '도시', '동', '사고유형_차대사람', '사고유형_차대차', '사고유형_차량단독', '노면상태_건조',
       '노면상태_기타', '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수',
       '도로형태1_교차로', '도로형태1_기타', '도로형태1_단일로', '도로형태1_미분류', '도로형태1_주차장',
       '도로형태2_고가도로위', '도로형태2_교량', '도로형태2_교차로부근', '도로형태2_교차로안',
       '도로형태2_교차로횡단보도내', '도로형태2_기타', '도로형태2_미분류', '도로형태2_주차장',
       '도로형태2_지하차도(도로)내', '도로형태2_터널', '구_남구', '구_달서구', '구_달성군', '구_동구', '구_북구',
       '구_서구', '구_수성구', '구_중구', '연_2019', '연_2020', '연_2021', '연_2022', '월_1',
       '월_2', '월_3', '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11',
       '월_12', '시간_0', '시간_1', '시간_2', '시간_3', '시간_4', '시간_5', '시간_6', '시간_7',
       '시간_8', '시간_9', '시간_10', '시간_11', '시간_12', '시간_13', '시간_14', '시간_15',
       '시간_16', '시간_17', '시간_18', '시간_19', '시간_20', '시간_21', '시간_22', '시간_23',
       '요일_월요일', '요일_화요일', '요일_수요일', '요일_목요일', '요일_금요일', '요일_토요일', '요일_일요일',
       '기상상태_기타', '기상상태_눈', '기상상태_맑음', '기상상태_비', '기상상태_안개', '기상상태_흐림'],
      dtype='object')

## **7. One-Hot Encoding 다중공선성 확인**

In [50]:
data_onehot = data_onehot_enco.drop(data_onehot_enco.columns[0], axis=1)

In [51]:
column_range = list(range(13))
data_corr = data_onehot.iloc[:, column_range]
data_corr

,일,도시,동,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,도로형태1_교차로
0,1,대구광역시,대신동,True,False,False,True,False,False,False,False,False,False
1,1,대구광역시,감삼동,True,False,False,True,False,False,False,False,False,False
2,1,대구광역시,두산동,True,False,False,True,False,False,False,False,False,False
3,1,대구광역시,복현동,False,True,False,True,False,False,False,False,False,False
4,1,대구광역시,신암동,False,True,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,31,대구광역시,대명동,False,True,False,True,False,False,False,False,False,False
10959,31,대구광역시,시지동,False,True,False,True,False,False,False,False,False,False
10960,31,대구광역시,연호동,False,True,False,True,False,False,False,False,False,False
10961,31,대구광역시,범물동,False,True,False,True,False,False,False,False,False,True


- feature간 상관계수를 통해 다중공선성 확인 결과, 뚜렷하게 나타나는 변수 간 강한 양의 상관관계는 없는 것으로 보아
- 다중공선성에 대한 문제는 없는 것으로 보임
- VIF(Variance Inflation Factors)를 통해 확인해보는 방법도 존재
- https://aliencoder.tistory.com/17 - VIF 설명

# **📌 최종 데이터**
1. **data_label_enco** : LabelEncoding 데이터프레임
2. **data_onehot_enco** : One-Hot Encoding 데이터프레임

In [52]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  int64 
 2   기상상태    50572 non-null  int64 
 3   노면상태    50572 non-null  int64 
 4   사고유형    50572 non-null  int64 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int64 
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int64 
 13  도로형태2   50572 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 5.8+ MB


In [53]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   일                50572 non-null  int64 
 2   도시               50572 non-null  object
 3   동                50572 non-null  object
 4   사고유형_차대사람        50572 non-null  bool  
 5   사고유형_차대차         50572 non-null  bool  
 6   사고유형_차량단독        50572 non-null  bool  
 7   노면상태_건조          50572 non-null  bool  
 8   노면상태_기타          50572 non-null  bool  
 9   노면상태_서리/결빙       50572 non-null  bool  
 10  노면상태_적설          50572 non-null  bool  
 11  노면상태_젖음/습기       50572 non-null  bool  
 12  노면상태_침수          50572 non-null  bool  
 13  도로형태1_교차로        50572 non-null  bool  
 14  도로형태1_기타         50572 non-null  bool  
 15  도로형태1_단일로        50572 non-null  bool  
 16  도로형태1_미분류        50572 non-null  bool  
 17  도로형태1_주차장        50572 non-null  boo

# **📌 모델링**

## **통합된 데이터 세트(data)를 다시 train과 test 세트로 분리**
- 합쳐둔 data 날짜에 맞춰서 다시 train_new, test_new로 분리하기

In [54]:
train_new  = data_onehot_enco[:len(train)]
test_new = data_onehot_enco[len(train):]

In [55]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39609 entries, 0 to 39608
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               39609 non-null  object
 1   일                39609 non-null  int64 
 2   도시               39609 non-null  object
 3   동                39609 non-null  object
 4   사고유형_차대사람        39609 non-null  bool  
 5   사고유형_차대차         39609 non-null  bool  
 6   사고유형_차량단독        39609 non-null  bool  
 7   노면상태_건조          39609 non-null  bool  
 8   노면상태_기타          39609 non-null  bool  
 9   노면상태_서리/결빙       39609 non-null  bool  
 10  노면상태_적설          39609 non-null  bool  
 11  노면상태_젖음/습기       39609 non-null  bool  
 12  노면상태_침수          39609 non-null  bool  
 13  도로형태1_교차로        39609 non-null  bool  
 14  도로형태1_기타         39609 non-null  bool  
 15  도로형태1_단일로        39609 non-null  bool  
 16  도로형태1_미분류        39609 non-null  bool  
 17  도로형태1_주차장        39609 non-null  boo

In [56]:
test_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10963 entries, 0 to 10962
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               10963 non-null  object
 1   일                10963 non-null  int64 
 2   도시               10963 non-null  object
 3   동                10963 non-null  object
 4   사고유형_차대사람        10963 non-null  bool  
 5   사고유형_차대차         10963 non-null  bool  
 6   사고유형_차량단독        10963 non-null  bool  
 7   노면상태_건조          10963 non-null  bool  
 8   노면상태_기타          10963 non-null  bool  
 9   노면상태_서리/결빙       10963 non-null  bool  
 10  노면상태_적설          10963 non-null  bool  
 11  노면상태_젖음/습기       10963 non-null  bool  
 12  노면상태_침수          10963 non-null  bool  
 13  도로형태1_교차로        10963 non-null  bool  
 14  도로형태1_기타         10963 non-null  bool  
 15  도로형태1_단일로        10963 non-null  bool  
 16  도로형태1_미분류        10963 non-null  bool  
 17  도로형태1_주차장        10963 non-null  boo

### (참고) 데이터 분할

- 주어진 데이터셋이 이미 훈련용(train)과 테스트용(test)으로 나뉘어져 있으므로 train_test_split 함수를 사용하여 데이터를 분할할 필요 없음!
- 그렇지만, 훈련데이터를 더 작은 훈련 세트(train set)와 검증 세트(validation set)로 나누는 것은 필요함.
> - train data set의 피쳐를 X, 타겟을 y로 분리
> - train data set를 훈련(train)과 검증(valid)로 나눈다.
> - train:valid= 7:3 로 나누어 본다
>> - 훈련 세트(train set): 모델을 훈련하는 데 사용됩니다.
>> - 검증 세트(validation set): 모델을 튜닝하고 성능을 평가하는 데 사용됩니다. 이를 통해 모델이 훈련 데이터에 과적합되지 않고 다른 데이터에 대해서도 일반화될 수 있는지 확인할 수 있습니다.
> - 대회에서는 주어진 테스트 데이터가 숨겨져 있으므로 모델을 평가하려면 모델의 예측을 검증 세트에 대해 평가하고, 최종적으로는 테스트 데이터에 대한 예측을 제출해야 합니다.

In [57]:
# from sklearn.model_selection import train_test_split 
# X_test1= train_new.drop('ID', axis=1)
# y_test1= train['ECLO']

In [58]:
# # train data set를 train과 validation data set로 나누기
# X_train, X_valid, y_train, y_valid = train_test_split(X_test1, y_test1, test_size=0.3, random_state=0)

In [59]:
# # 데이터 사이즈 확인
# print("Size of X_train is:{}\nSize of y_train is:{}\nSize of X_valid is:{}\nSize of Y_valid is:{}\n".format(
#     X_train.shape,y_train.shape,X_valid.shape,y_valid.shape))

## **피처에 대한 모델링 진행 방법**
- 아래 방법 중 원하는 방법으로 사용
1. 모든 columns (features) 사용하기 (ID, 도시, 동 제외한 모든 columns)
2. 학습시킬 column (feature) 선택적으로 사용하기
3. 참고사항 (RMSLE 함수, Submission 형식)

### **1. 모든 columns (features) 사용하기**

In [60]:
# # ID, 도시, 동 제외한 모든 컬럼
# X_test = test_new.drop(columns=['ID','도시','동']).copy()
# X_train = train_new[X_test.columns].copy()

In [61]:
# y_train = train['ECLO'].copy()

### **2. 학습시킬 column (feature) 선택적으로 사용하기**

In [62]:
# # 예측에 넣을 feature 추가하기
# feature =['요일','기상상태','노면상태','사고유형','연','월','시간',,,,,,] 

In [63]:
# X_train = train_new[feature]
# X_test = test_new[feature]

In [64]:
# y_train = train['ECLO'].copy()

### **3. 참고 사항**

- 변수 이름 선택한 모델명으로 설정하기!!!

In [65]:
# 예시

# from sklearn.tree import DecisionTreeRegressor
# dtr = DecisionTreeRegressor()
# dtr.fit(X_train, y_train)

# prediction_dtr= dtr.predict(X_test)
# prediction_dtr

- 평가함수 (RMSLE)
  - 리더보드 제출 전 RMSLE 점수 계산해볼 수 있는 코드입니다. 실제 점수와는 조금 다를수 있어요!

In [66]:
# from sklearn.metrics import mean_squared_log_error
# # RMSLE 계산
# rmsle = mean_squared_log_error(y_test, y_pred, squared=False)

# # 결과 출력
# print(f'RMSLE: {rmsle}')

In [67]:
# import numpy as np
# from sklearn.metrics import make_scorer

# def rmsle(predict, actual):
#     log_predict= np.log(predict +1)
#     log_actual = np.log(actual+1)
    
#     difference = log_predict - log_actual
#     difference = np.square(difference)
    
#     mean_difference = difference.mean()
    
#     score = np.sqrt(mean_difference)
    
#     return score

# rmsle_score = make_scorer(rmsle)

In [68]:
# from sklearn.model_selection import cross_val_score

# score = cross_val_score(model , X_train, y_train, cv =5, 
#                         scoring=rmsle_score).mean()

# print('현재 점수는 {0:.5f}'.format(score))

- Submission
  - 형식 복붙해서 사용하시는 모델에 맞춰서 저장해주세요!

In [69]:
# A (A or B choose)

In [70]:
# sample_submission = pd.read_csv('open/sample_submission.csv')
# sample_submission

In [71]:
# B (A or B choose)

In [72]:
# sample_submission = pd.read_csv('open/sample_submission.csv', index_col=0)
# sample_submission

In [73]:
# prediction_모델이름

In [74]:
# 모델이름_submission = sample_submission.copy()
# 모델이름_submission['ECLO'] = prediction_모델이름
# 모델이름_submission

In [75]:
# 모델이름_submission.to_csv('모델이름_submission.csv')

## **🟣 모델링 진행 (검증)**

- 선택한 모델로 test 결과 저장하기
- 검증 데이터세트의 정확도가 높은 모델을 채택!

### **LightGBM - 검증**
- RMSLE: 0.466, RMSE: 3.133, MAE: 2.158

- Packages

In [76]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

- 데이터 준비 (train data를 train,val 로 나누기)

In [77]:
# 훈련 데이터에서 목표 변수 추출
target_column = "ECLO"
y_train = train[target_column]

# 훈련 데이터에서 불필요한 열 제거
features = train_new.drop(["ID", "도시", "동"], axis=1)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(features, y_train, test_size=0.3)

- LightGBM 학습

In [78]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)

LGBMRegressor()

- 예측

In [79]:
pred_lgbm = lgbm.predict(X_val)

- 평가

In [80]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
#  1. RMSLE 작성
def rmsle(pred, actual):
    log_pred = np.log1p(pred) 
    log_actual = np.log1p(actual)
    squared_error = (log_pred-log_actual)**2   
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle
#  2. RMSE 작성
def rmse(pred, actual):
    return np.sqrt(mean_squared_error(pred, actual))
#  3. MAE 작성
def mae(pred, actual):
    return mean_absolute_error(pred, actual)

In [81]:
def evaluate_regr(pred, actual):
    rmsle_val=rmsle(pred, actual)
    rmsl_val=rmse(pred, actual)
    mae_val=mae(pred, actual)
    print('RMSLE: {:.3f}, RMSE: {:.3f}, MAE: {:.3f}'.format(rmsle_val, rmsl_val, mae_val))

In [82]:
evaluate_regr(pred_lgbm, y_val)

RMSLE: 0.466, RMSE: 3.133, MAE: 2.158


### **LightGBM - 최적의 하이퍼파라미터 찾기 (Grid Search)**
- RMSLE: 0.453, RMSE: 3.092, MAE: 2.107

In [83]:
# from sklearn.model_selection import GridSearchCV

In [84]:
# # Hyperparameter Grid 정의 - 각 parmas의 주요 range 에서 초기 범위 적용
# param_grid = {
#     'num_leaves': [20, 30, 40],             # Typical : 20~100
#     'learning_rate': [0.005, 0.01, 0.1],    # Typical : 0.005 ~ 0.2
#     'n_estimators': [50, 100, 200],         # Typical : 50 ~ 1000
#     'max_depth': [5, 10, 15],               # Typical 3 ~ 15
#     'feature_fraction' : [0.5, 0.75, 1]     # Typical 0.5 ~ 1
# }

In [85]:
# # LGBM 모델 생성
# lgbm = lgb.LGBMRegressor()

In [86]:
# # GridSearchCV 객체 생성
# grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

In [87]:
# # 데이터에 grid_search 적용
# grid_search.fit(X_train, y_train)

In [88]:
# # best hyperparameters 는?
# print("Best Hyperparameters:", grid_search.best_params_)

In [89]:
# # model 에 적용 
# best_model = grid_search.best_estimator_

In [90]:
# print("Best Models:", best_model)

In [91]:
# # 예측 
# pred_grid = best_model.predict(X_val)

In [92]:
# # 평가 함수 (1)
# from sklearn.metrics import mean_squared_error,mean_absolute_error
# #  1. RMSLE 작성
# def rmsle(pred, actual):
#     log_pred = np.log1p(pred) 
#     log_actual = np.log1p(actual)
#     squared_error = (log_pred-log_actual)**2   
#     rmsle = np.sqrt(np.mean(squared_error))
#     return rmsle
# #  2. RMSE 작성
# def rmse(pred, actual):
#     return np.sqrt(mean_squared_error(pred, actual))
# #  3. MAE 작성
# def mae(pred, actual):
#     return mean_absolute_error(pred, actual)

In [93]:
# # 평가 함수 (2)
# def evaluate_regr(pred, actual):
#     rmsle_val=rmsle(pred, actual)
#     rmsl_val=rmse(pred, actual)
#     mae_val=mae(pred, actual)
#     print('RMSLE: {:.3f}, RMSE: {:.3f}, MAE: {:.3f}'.format(rmsle_val, rmsl_val, mae_val))

In [94]:
# # 평가
# evaluate_regr(pred_grid, y_val)

### **LightGBM - 최적의 하이퍼파라미터 찾기 (Randomized Search)**
- RMSLE: 0.445, RMSE: 3.035, MAE: 2.071

In [95]:
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform, randint

In [96]:
# # Hyperparameter Distributions 정의 - 각 parmas의 주요 range 에서 최소/최대값 적용 
# param_dist = {
#     'num_leaves': randint(20, 100),          # Typical : 20~100
#     'learning_rate': uniform(0.005, 0.2),    # Typical : 0.005 ~ 0.2
#     'n_estimators': randint(50, 1000),       # Typical : 50 ~ 1000
#     'max_depth': randint(3, 15),             # Typical 3 ~ 15
# }

In [97]:
# # LGBM 모델 생성
# lgbm = lgb.LGBMRegressor()

In [98]:
# # RandomizedSearch CV 객체 생성
# random_search = RandomizedSearchCV(estimator=lgbm, param_distributions=param_dist, scoring='neg_mean_squared_error', n_iter=10, cv=3)

In [99]:
# # 데이터에 random_search 적용
# random_search.fit(X_train, y_train)

In [100]:
# # best hyperparameters 는?
# print("Best Hyperparameters:", random_search.best_params_)

In [101]:
# # model 에 적용 
# best_model = random_search.best_estimator_

In [102]:
# print("Best Models:", best_model)

In [103]:
# # 예측 
# pred_random = best_model.predict(X_val)

In [104]:
# # 평가
# evaluate_regr(pred_random, y_val)

## **🟣 원본 데이터에 모델 적용 (Randomized Search Hyperparams 사용)**
- Best Hyperparameters: {'learning_rate': 0.056307085566180866, 'max_depth': 10, 'n_estimators': 131, 'num_leaves': 43}

- 데이터 준비

In [105]:
# 예측에 모든 컬럼 사용하는 법.
X_test = test_new.drop(columns=['ID','도시','동']).copy()
X_train = train_new[X_test.columns].copy()

In [106]:
y_train = train['ECLO'].copy()

In [107]:
sample_submission = pd.read_csv('/Users/euijinlee/KDT_DATA/Project_1/data/sample_submission.csv', index_col=0)
sample_submission

,ECLO
ID,
ACCIDENT_39609,0
ACCIDENT_39610,0
ACCIDENT_39611,0
ACCIDENT_39612,0
ACCIDENT_39613,0
...,...
ACCIDENT_50567,0
ACCIDENT_50568,0
ACCIDENT_50569,0


In [108]:
#y_test로 일단 데이터수 확인 위해 설정
y_test = sample_submission['ECLO'].copy()

In [109]:
# 데이터 사이즈 확인
print("Size of X_train is:{}\nSize of y_train is:{}\nSize of X_test is:{}\nSize of y_test is:{}\n".format(
    X_train.shape,y_train.shape,X_test.shape,y_test.shape))

Size of X_train is:(39609, 86)
Size of y_train is:(39609,)
Size of X_test is:(10963, 86)
Size of y_test is:(10963,)



- LightGBM 진행
  - Randomized Search 에서 찾은 최적의 하이퍼 파라미터 사용
    - Best Hyperparameters: {'learning_rate': 0.056307085566180866, 'max_depth': 10, 'n_estimators': 131, 'num_leaves': 43}

In [110]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [111]:
best_params = {
    'learning_rate': 0.056307085566180866,
    'max_depth': 10,
    'n_estimators': 131,
    'num_leaves': 43
}

In [112]:
best_lgbm_model = lgb.LGBMRegressor(**best_params)

In [113]:
best_lgbm_model.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.056307085566180866, max_depth=10,
              n_estimators=131, num_leaves=43)

In [114]:
pred_lgbm = best_lgbm_model.predict(X_test)

In [117]:
LGBM_submission = sample_submission.copy()
LGBM_submission['ECLO'] = pred_lgbm
LGBM_submission

,ECLO
ID,
ACCIDENT_39609,3.886096
ACCIDENT_39610,3.650136
ACCIDENT_39611,5.337318
ACCIDENT_39612,5.076716
ACCIDENT_39613,5.012273
...,...
ACCIDENT_50567,5.546434
ACCIDENT_50568,4.568139
ACCIDENT_50569,4.750883


In [118]:
#submission 파일로 저장
LGBM_submission.to_csv('LGBM_submission.csv')

# ➡️ 대회 제출 점수 : 0.4423470594